<a href="https://colab.research.google.com/github/Umesh9045/LP5/blob/main/cuda_matrix_multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ekivt84_
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ekivt84_
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-vp5yriw5/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
pip install nvcc4jupyter

In [4]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp33ieu_za".


In [7]:
%%cuda

#include <iostream>

using namespace std;

__global__ void matrixMultiplication(int *a, int *b, int *c) {
    int row = threadIdx.y;
    int col = threadIdx.x;

    // Perform matrix multiplication for 2x2 matrices
    c[row * 2 + col] = a[row * 2] * b[col] + a[row * 2 + 1] * b[2 + col];
}

int main() {
    // Define the matrices
    int a[2][2] = {{1, 2}, {3, 4}};
    int b[2][2] = {{5, 6}, {7, 8}};
    int c[2][2]; // Result matrix

    // cudaMalloc function is used allocate memory on the GPU (device memory)
    int *dev_a, *dev_b, *dev_c;
    cudaMalloc((void**)&dev_a, sizeof(int) * 2 * 2);
    cudaMalloc((void**)&dev_b, sizeof(int) * 2 * 2);
    cudaMalloc((void**)&dev_c, sizeof(int) * 2 * 2);

    // Copy matrices from host to device
    cudaMemcpy(dev_a, a, sizeof(int) * 2 * 2, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, sizeof(int) * 2 * 2, cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 threadsPerBlock(2, 2);
    matrixMultiplication<<<1, threadsPerBlock>>>(dev_a, dev_b, dev_c);

    // Copy result matrix from device to host
    cudaMemcpy(c, dev_c, sizeof(int) * 2 * 2, cudaMemcpyDeviceToHost);

    // Print the result matrix
    cout << "Resultant Matrix:" << endl;
    for (int i = 0; i < 2; ++i) {
        for (int j = 0; j < 2; ++j) {
            cout << c[i][j] << " ";
        }
        cout << endl;
    }

    // Free device memory
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}


Resultant Matrix:
19 22 
43 50 

